In [ ]:
import pandas as pd
from pyomo.environ import *
# from itertools import product
from clean_data import *
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# {start} chamar dados
demanda = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/demanda.csv')
preco = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/preco.csv')
rota = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/instancias/rota.csv')
# {end} chamar dados

origin1, destin, oridest, vagones, periodos, stations, clases, preco, dem= clean_data(demanda, preco)
origin2 = [0] + origin1
n = len(stations)
cap = 561
rota = eval(rota['Route'][0])

In [ ]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.Q = Param(initialize = cap)

# conjuntos
model.I2 = Set(initialize = origin2)
model.J = Set(initialize = destin)
model.OD = Set(dimen=2, initialize = oridest)
model.V = Set(initialize = vagones)
model.T = Set(initialize = periodos)
model.VK = Set(model.V, initialize=clases)
# model.K = Set(initialize = clases)
# model.I1 = Set(initialize = origin1)
def wagons_class(model):
    return ((v,k) for v in clases.keys() for k in clases[v])
model.VKI = Set(dimen=2,initialize=wagons_class)


#parâmetros indexados
model.P = Param(model.OD, model.VKI, initialize = preco)
model.d = Param(model.OD, model.VKI, model.T, initialize = dem)

# variáveis de decisão
model.X = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.Y = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.A = Var(model.I2, within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,v,k]*(model.X[i,j,v,k,t]) for i,j in model.OD for v,k in model.VKI for t in model.T)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def cap1(model, i,_):
    return model.A[rota.index(i)+1] ==  model.A[rota.index(i)+1-1] - sum((model.X[i-1,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if rota.index(j)>=rota.index(i)) + sum((model.X[j,i,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if rota.index(j)<rota.index(i))
model.ConstrainCap1 = Constraint(model.OD, rule = cap1)

def cap2(model, i,_):
    if rota.index(i) < model.n:
        return sum((model.X[i,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T) <= model.A[rota.index(i)+1]
    else:
        return Constraint.Skip
model.ConstrainCap2 = Constraint(model.OD, rule = cap2)

def authorizationCap(model):
    return  sum(model.Y[i,j,v,list(model.VK[v])[0],t] for i,j in model.OD for v in model.VK for t in model.T) <= model.Q
model.ConstrainAuthorizationCap = Constraint(rule = authorizationCap)

def authorization(model, i, j, v, k):
    lista = list(model.VK[v])
    pos_k = lista.index(k)
    lista_sort = sorted(lista, reverse=True)
    if pos_k == 0:
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    elif pos_k <= len(lista)-1 :
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.Y[i,j,v,lista_sort[pos_k-1],t] for t in model.T) + sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)

# def authorization(model, i, j, v, k):
#     lista = list(model.VK[v])
#     if i < j:
#         pos_k = lista.index(k)
#         sub_list = lista[pos_k:]
#         return  sum(model.Y[i,j,v,k,t] for t in model.T) >= sum(model.X[i,j,v,r,t] for r in sub_list for t in model.T )
#     else:
#         return Constraint.Skip
# model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)


def classe(model, i, j, v, k, t):
    lista = list(model.VK[v])
    if k != lista[-1]:
        pos_k = lista.index(k)
        return model.Y[i,j,v,k,t] >= model.Y[i,j,v,lista[pos_k+1],t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.OD, model.VKI, model.T, rule = classe)

def assignment(model, i, j, v, k, t):
    if rota.index(i) < rota.index(j):
        return model.X[i,j,v,k,t] <= model.d[i,j,v,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.OD, model.VKI, model.T,rule = assignment)

def assignment0(model, _, j, v, k, t):
    return model.X[0,j,v,k, t] == 0
model.ConstrainAssignment0 = Constraint(model.OD, model.VKI, model.T, rule = assignment0)

def authorization0(model, _, j, v, k, t):
    return model.Y[0,j,v,k,t] == 0
model.ConstrainAuthorization0 = Constraint(model.OD, model.VKI, model.T, rule = authorization0)

def cap0(model):
    return model.A[0] == model.Q
model.ConstrainCap0 = Constraint(rule = cap0)

# def class0(model, i, j, v, k):
#     return model.X[i,j,v,k,0] == 0
# model.ConstrainClass0 = Constraint(model.OD, model.VKI, rule = class0)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
# print(results)

In [ ]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i,j in instance.OD:
    for v,k in instance.VKI:
        for t in instance.T:
            # if value(instance.Y[i,j,v,k,t])!=0 or value(instance.X[i,j,v,k,t])!=0:
                # print(i,j,v,k,t, ' - ',value(instance.Y[i,j,v,k,t]), ' - ', value(instance.X[i,j,v,k,t]) )
            lista.append([str(i) + '-' + str(j),v, k, t, value(instance.P[i,j,v,k]), value(instance.d[i,j,v,k,t]) , value(instance.X[i,j,v,k,t]), value(instance.Y[i,j,v,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','Vagon','classe','Periodo','preco','demanda','Assignments','Authorizations'])
a.to_excel('prueba_m1_2.xlsx', index=False)

In [ ]:
# dem = {(i,j,v,k,t): np.random.randint(1,100) for i,j in oridest for v in vagones for k in clases for t in periodos }

# preco = {}
# for i,j in oridest:
#     for v in vagones:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(clases)), reverse=True)
#         for k,valor in enumerate(aleat):
#             preco[(i,j,v,clases[k])] = valor


# oridest = [(1,2),(1,3),(1,4),(1,5)]
# origin1 = [1]
# origin2 = [0,1]
# destin = [2,3,4,5]
# vagones = [1,2]
# periodos = [0,1,5,10]

# cap = 200 
# n = 5 
# clases = {1:[1,5,7],2:[1,5,7]}
# # clases = [1,5,7]

# corrigir instancias
# dem = {(i,j,v,k,t): dem[(i,j,v,k,t)] if (i,j,v,k,t) in dem else 0  for i,j in oridest for v in vagones for k in clases_ord[v] for t in periodos}
# preco = {(i,j,v,k): preco[(i,j,v,k)] if (i,j,v,k) in preco else 0  for i,j in oridest for v in vagones for k in clases_ord[v] }